In [ ]:
#!/usr/bin/env python
# coding: utf-8
from __future__ import division, print_function

import sys, os
sys.path.append('..')

import theano
import theano.tensor as T
import lasagne

import time
import argparse

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
!pip install seaborn;
import seaborn as sns; sns.set()

from logs import log_fname, new_logger
from nn.rgl import ReverseGradientLayer
from nn.block import Dense, Classifier
from nn.compilers import crossentropy_sgd_mom, squared_error_sgd_mom
from nn.training import Trainner, training

from utils import plot_bound, save_confusion_matrix
from sklearn.metrics import confusion_matrix

In [ ]:
%matplotlib inline

# Load datasets

Here the datasets are loaded.

## Clouds


### Clouds rotated

In [ ]:
from datasets.toys import load_cloud_rotated


data_name = 'Clouds_Rotated'
n_samples = 30  # Number of sample per class
n_classes = 12
batchsize = 80
angle = 80

source_data, target_data, domain_data = load_cloud_rotated(n_sample=n_samples, 
                                                           n_classes=n_classes, 
                                                           angle=angle, 
                                                           batchsize=batchsize)


### Clouds . Random Matrix

In [ ]:
from datasets.toys import load_cloud_rotated
from datasets.utils import random_mat_dataset


data_name = 'Clouds_RMat'
n_samples = 30  # Number of sample per class
n_classes = 12
batchsize = 80
angle = 80

source_data, target_data, domain_data = load_cloud_rotated(n_sample=n_samples, 
                                                           n_classes=n_classes, 
                                                           angle=angle, 
                                                           batchsize=batchsize)
source_data, target_data, domain_data = random_mat_dataset(source_data)


## Moon


### Moon rotated

In [ ]:
from datasets.toys import load_moon

data_name = 'Moon_Rotated'
n_samples = 800
batchsize = 80
angle = 30.

source_data, target_data, domain_data = load_moon(n_samples=n_samples, angle=angle, batchsize=batchsize)


### Moon . Random Matrix

In [ ]:
from datasets.toys import load_moon
from datasets.utils import random_mat_dataset

data_name = 'Moon_RMat'
n_samples = 800
batchsize = 80
angle = 30.

source_data, target_data, domain_data = load_moon(n_samples=n_samples, angle=angle, batchsize=batchsize)
source_data, target_data, domain_data = random_mat_dataset(source_data)


## MNIST

### MNIST . Diag Dominant matrix

In [ ]:
from datasets.mnist import load_mnist_src
from datasets.utils import diag_dataset

data_name = 'MNIST_Diag'
batchsize = 500

source_data = load_mnist_src(batchsize=batchsize)
source_data, target_data, domain_data = diag_dataset(source_data, normalize=True)


### MNIST Mirror

In [ ]:
from datasets.mnist import load_mnist_mirror

data_name = 'MNIST_Mirror'
batchsize = 500

source_data, target_data, domain_data = load_mnist_mirror(batchsize=batchsize)


### MNIST . Random Matrix

In [ ]:
from datasets.mnist import load_mnist_src
from datasets.utils import random_mat_dataset

data_name = 'MNIST_Rmat'
batchsize = 500

source_data = load_mnist_src(batchsize=batchsize)
source_data, target_data, domain_data = random_mat_dataset(source_data, normalize=True)


## Make the *corrector* and *domain* dataset.

In [ ]:
domain_data = {
            'X_train': [source_data['X_train'], target_data['X_train']],
            'X_val': [source_data['X_val'], target_data['X_val']],
            'X_test': [source_data['X_test'], target_data['X_test']],
            'y_train': None,
            'y_val': None,
            'y_test': None,
            'batchsize': batchsize,
            }    

corrector_data = dict(target_data)
corrector_data.update({
    'y_train': source_data['X_train'],
    'y_val': source_data['X_val'],
    'y_test': source_data['X_test'],
    'labels': source_data['y_train'],
    'batchsize': batchsize,
    })


# Build the Neural Network

In [ ]:
# Get a logger
logger = new_logger()

## Parameters

In [ ]:
num_epochs = 5

hp_lambda = 0.0

label_rate = 1
label_mom = 0.9

domain_rate = 1
domain_mom = 0.9

k = 5

## Epoch Preprocessing

The preprocessing function that will run at the begining of each epoch

In [ ]:
# http://stackoverflow.com/questions/25886374/pdist-for-theano-tensor
# Tested and approved
X = T.fmatrix('X')
Y = T.fmatrix('Y')
translation_vectors = X.reshape((X.shape[0], 1, -1)) - Y.reshape((1, Y.shape[0], -1))
euclidiean_distances = (translation_vectors ** 2).sum(2)
f_euclidean = theano.function([X, Y], euclidiean_distances, allow_input_downcast=True)

In [ ]:
def kclosest(X, Y, k, batchsize=None):
    """
    Computes for each sample from X the k-closest samples in Y and return 
    their index.

    Params
    ------
        X: (numpy array [n_sample, n_feature])
        Y: (numpy array [n_sample, n_feature])
        k: (int)
    Return
    ------
        kclosest : (numpy array [n_sample, k]) the ordered index of 
            the k-closest instances from Y to X samples
    """
    assert X.shape == Y.shape
    N = X.shape[0]
    if batchsize is None:
        dist = f_euclidean(X, Y)
    else:
        dist = np.empty((N, N), dtype=theano.config.floatX)
        batch = np.arange(0, N+batchsize, batchsize)
        for excerpt_X in (slice(i0, i1) for i0, i1 in zip(batch[:-1], batch[1:])):
            dist[excerpt_X] = f_euclidean(X[excerpt_X], Y)
    kbest = np.argsort(dist, axis=1)[:, :k]
    return kbest

In [ ]:
def realign(X_out, X_trg, y, k=20, batchsize=None):
    counter = np.zeros(X_out.shape[0], dtype=int)
    idx = np.empty_like(y, dtype=int)
    for label in np.unique(y):
        # Get the examples of the right label
        idx_label = np.where(y==label)[0]

        # Get the k-closest index ... shape = ... ça va pas du tout !
        idx_label2 = kclosest(X_out[idx_label], X_trg[idx_label], k, batchsize=batchsize)
        
        for i1, i2 in zip(idx_label, idx_label2):
            # i2 is an index array of shape (k,) with the sorted closest example index 
            # (of the sorted single class array)
            # Then idx_label[i2] are the sorted original index of the k-closest examples
            i = idx_label[i2[np.argmin(counter[idx_label[i2]])]]
            # i contains the chosen one, in the (k-)clostest example, with the minimum counter
            counter[i] = counter[i]+1
            idx[i1] = i
    return idx

In [ ]:
def batchpad(batchsize, output_shape, dtype=None):
    """Re-batching decorator
    """
    def decoreted(func):
        def wrapper(X, *args, **kwargs):
            if dtype is None:
                dtype2 = X.dtype
            else:
                dtype2 = dtype
            
            N = X.shape[0]
            
            if output_shape is None:
                shape = X.shape
            else:
                shape = tuple( out_s if out_s is not None else X_s for out_s, X_s in zip(output_shape, X.shape))

            result = np.empty(shape, dtype=dtype2)
            batch = np.arange(0, N+batchsize, batchsize)
            for excerpt_X in (slice(i0, i1) for i0, i1 in zip(batch[:-2], batch[1:])):
                result[excerpt_X] = func(X[excerpt_X], *args, **kwargs)
            
            last_excerpt = slice(batch[-2], batch[-1])
            X = X[last_excerpt]
            n_sample = X.shape[0]
            X = np.vstack([X, np.zeros((batchsize-X.shape[0],)+X.shape[1:])])
            X = func(X, *args, **kwargs)
            result[last_excerpt] = X[:n_sample]
            
            return result
        return wrapper
    return decoreted

In [ ]:
def preprocess(data, trainer, epoch):
    X = data['X_train']

    @batchpad(data['batchsize'], X.shape, X.dtype)
    def f_output(X, trainer):
        return trainer.output(X)[0]
    
    X_out = f_output(X, trainer)
    X_trg = data['y_train']
    data['X_train'] = X[realign(X_out, X_trg, data['labels'], k=5, batchsize=None)]

In [ ]:
def classwise_shuffle(X, y):
    """
    Shuffle X without changing the class positions

    Params
    ------
        X: the data (numpy array)
        y: the labels 
    Return
    ------
        X_shuffled: Shuffled X without changing the class matching
    """
    idx = np.empty_like(y, dtype=int)
    for label in np.unique(y):
        arr = np.where(y==label)[0]
        arr2 = np.random.permutation(arr)
        idx[arr] = arr2
    return X[idx]


def epoch_shuffle(data, trainer, epoch, *args, **kwargs):
    data['X_train'] = classwise_shuffle(data['X_train'], data['labels'])
    return data


## Network building
Start with the variables

In [ ]:
# Prepare Theano variables for inputs and targets
if data_name.startswith('MNIST'):
    input_var = T.tensor3('inputs')
    src_var = T.tensor3('src')
    target_var = T.tensor3('targets')
    shape = (batchsize, 28, 28)
elif data_name.startswith('Moon') or data_name.startswith('Clouds'):
    input_var = T.matrix('inputs')
    src_var = T.matrix('src')
    target_var = T.matrix('targets')
    shape = (batchsize, 2)

# Logs
logger.info('Building the input and output variables for |{}|'.format(data_name))
logger.info('Input data expected shape : {}'.format(shape))


Build the neual network architecture

In [ ]:
# Build the layers
input_layer = lasagne.layers.InputLayer(shape=shape, input_var=input_var)
src_layer = lasagne.layers.InputLayer(shape=shape, input_var=src_var)
# feature = lasagne.layers.DenseLayer(
#                 input_layer,
#                 num_units=np.prod(shape[1:]),
#                 nonlinearity=lasagne.nonlinearities.tanh,
#                 # W=lasagne.init.Uniform(range=0.01, std=None, mean=0.0),
#                 )
feature = lasagne.layers.DenseLayer(
                input_layer,
                num_units=np.prod(shape[1:]),
                nonlinearity=None,
                # W=lasagne.init.Uniform(range=0.01, std=None, mean=0.0),
                )
reshaper = lasagne.layers.ReshapeLayer(feature, (-1,) + shape[1:])
output_layer = reshaper

# Logs
logger.info('Building the neural network architecture for |{}|'.format(data_name))
logger.info('Input data expected shape : {}'.format(shape))


Compiling the neural network

In [ ]:
# Logs
logger.info('Compiling the neural network for |{}|'.format(data_name))
logger.info('Input data expected shape : {}'.format(shape))

# Compilation
corrector_trainner = Trainner(squared_error_sgd_mom(output_layer, lr=label_rate, mom=0, target_var=target_var), 
                             'corrector',)

if hp_lambda != 0.0:
    print('hp_lambda != 0 : Compliling the adversarial part of the networks')
    domain_trainner = Trainner(adversarial([src_layer, output_layer], hp_lambda=hp_lambda,
                                          lr=domain_rate, mom=domain_mom),
                               'domain')


Add preprocessing (for alignment)

In [ ]:
# Add preprocessing :
corrector_trainner.preprocess = epoch_shuffle
#corrector_trainner.preprocess = preprocess

#model_name = 'Pairwise_Corrector'
model_name = 'Classwise_Corrector'
#model_name = 'K-closest_Corrector'

# Train the neural network

Reset the counter and the stats

In [ ]:
logger.warn('Reset the epoch counter and saved statistics')
epoch_counter = 0
final_stats = {}


## Training loop 

In [ ]:
n_epoch = 1
epoch_counter += n_epoch
logger.info('Trainning the neural network for {} additional epochs ({} total)'.format(n_epoch, epoch_counter))
if hp_lambda != 0.0:
    stats = training([corrector_trainner, domain_trainner], [corrector_data, domain_data],
                     num_epochs=n_epoch, logger=logger)
else:
    stats = training([corrector_trainner,], [corrector_data,],
                 num_epochs=n_epoch, logger=logger)

final_stats = {k: (final_stats[k]+v if k in final_stats else v) for k, v in stats.items()}

# Plot results

In [ ]:
title = '++'.join([data_name, model_name, 'lambda={:.3e}'.format(hp_lambda)])

## Learning curve


In [ ]:
# Plot learning accuracy curve
fig, ax = plt.subplots()
ax.plot(final_stats['corrector valid loss'], label='source')
ax.set_xlabel('epoch')
ax.set_ylabel('loss')
ax.set_title(title)
handles, labels = ax.get_legend_handles_labels()
ax.legend(handles, labels, bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
#fig.savefig('fig/'+title+'.png', bbox_inches='tight')
#fig.clf() # Clear plot window
plt.show()

## 2D Data plot

In [ ]:
from matplotlib.colors import ListedColormap
import matplotlib.cm as cm
cm_bright = ListedColormap(['#FF0000', '#0000FF'])
if data_name.startswith('Moon'):
    color = cm.ScalarMappable(cmap=cm_bright)
else:
    color = cm.ScalarMappable(cmap='Paired')

if data_name.startswith('Moon') or data_name.startswith('Clouds'):
    # Plot the test data
    fig, ax = plt.subplots()
    X = source_data['X_test']
    y = source_data['y_test']
    ax.scatter(X[:, 0], X[:, 1], label='source', marker='o', s=80, edgecolors=color.to_rgba(y), facecolors='none')

    X = np.array(corrector_trainner.output(target_data['X_test'])).reshape((-1, 2))
    y = target_data['y_test']
    ax.scatter(X[:, 0], X[:, 1], label='corrected', marker='x', s=80, edgecolors=color.to_rgba(y), facecolors=color.to_rgba(y))
    ax.set_title(title)
    handles, labels = ax.get_legend_handles_labels()
    ax.legend(handles, labels, bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
    #fig.savefig('fig/'+title+'-corrected_data.png', bbox_inches='tight')
    plt.show()

    logger.info('Data plot {}'.format(X.shape))


### Code concatenation !

In [ ]:
n_epoch = 10
epoch_counter += n_epoch
logger.info('Trainning the neural network for {} additional epochs ({} total)'.format(n_epoch, epoch_counter))
if hp_lambda != 0.0:
    stats = training([corrector_trainner, domain_trainner], [corrector_data, domain_data],
                     num_epochs=n_epoch, logger=logger)
else:
    stats = training([corrector_trainner,], [corrector_data,],
                 num_epochs=n_epoch, logger=logger)

final_stats = {k: (final_stats[k]+v if k in final_stats else v) for k, v in stats.items()}

# -----
title = '++'.join([data_name, model_name, 'lambda={:.3e}'.format(hp_lambda)])
# Plot learning accuracy curve
fig, (ax, ax1) = plt.subplots(1, 2, figsize=(20,10))
ax.plot(final_stats['corrector valid loss'], label='source')
ax.set_xlabel('epoch')
ax.set_ylabel('loss')
ax.set_title(title)
handles, labels = ax.get_legend_handles_labels()
ax.legend(handles, labels, bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
#fig.savefig('fig/'+title+'.png', bbox_inches='tight')
#fig.clf() # Clear plot window
#plt.show()

# -----
from matplotlib.colors import ListedColormap
import matplotlib.cm as cm
cm_bright = ListedColormap(['#FF0000', '#0000FF'])
if data_name.startswith('Moon'):
    color = cm.ScalarMappable(cmap=cm_bright)
else:
    color = cm.ScalarMappable(cmap='Paired')

if data_name.startswith('Moon') or data_name.startswith('Clouds'):
    # Plot the test data
    #fig, ax = plt.subplots()
    X = source_data['X_test']
    y = source_data['y_test']
    ax1.scatter(X[:, 0], X[:, 1], label='source', marker='o', s=80, edgecolors=color.to_rgba(y), facecolors='none')

    X = np.array(corrector_trainner.output(target_data['X_test'])).reshape((-1, 2))
    y = target_data['y_test']
    ax1.scatter(X[:, 0], X[:, 1], label='corrected', marker='x', s=80, edgecolors=color.to_rgba(y), facecolors=color.to_rgba(y))
    ax1.set_title(title)
    handles, labels = ax1.get_legend_handles_labels()
    ax1.legend(handles, labels, bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
    #fig.savefig('fig/'+title+'-corrected_data.png', bbox_inches='tight')
    fig.show()

    logger.info('Data plot {}'.format(X.shape))
